# GBM Example


This notebook implements the OmicsMind pipeline for GBM dataset:
- `rna.csv`
- `rppa.csv`
- `methylation.csv`


In [1]:
# !pip -q install pandas numpy scikit-learn tqdm torch

import os, random, warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from omicsmind.data import load_omics_table, MultiOmicsDataset, compute_nrmse_numpy
from omicsmind.omicsmind import ModalityVAE, CrossOmicsTransformer, OmicsMind, omicsmind_loss

print("torch:", torch.__version__)


torch: 2.9.1+cpu


## 1. Load data tables

In [2]:
data_dir = "./data/GBM_train"   
files = {
    "rna": "rna.csv",
    "rppa": "rppa.csv",
    "meth": "meth.csv",
}

modalities = {}
missing_files = []
for k, fn in files.items():
    p = os.path.join(data_dir, fn)
    if os.path.exists(p):
        modalities[k] = load_omics_table(p)
        print(k, "loaded", modalities[k].shape)
    else:
        missing_files.append(p)

if missing_files:
    print("\n[WARN] Missing files, will use synthetic data:", missing_files)


rna loaded (32, 19660)
rppa loaded (32, 158)
meth loaded (32, 20114)


## 2. If files are missing, generate synthetic data(Optional)


In [3]:
if missing_files:
    np.random.seed(0)
    n = 80
    modalities = {
        "rna":  pd.DataFrame(np.random.randn(n, 2000), index=[f"S{i:03d}" for i in range(n)]),
        "rppa": pd.DataFrame(np.random.randn(n, 200),  index=[f"S{i:03d}" for i in range(n)]),
        "meth": pd.DataFrame(np.random.randn(n, 1000), index=[f"S{i:03d}" for i in range(n)]),
    }
    # Simulate block-wise missingness
    idx = modalities["rna"].index
    mask_rppa = np.random.choice(idx, size=int(0.3*n), replace=False)
    mask_meth = np.random.choice(idx, size=int(0.2*n), replace=False)
    modalities["rppa"].loc[mask_rppa] = np.nan
    modalities["meth"].loc[mask_meth] = np.nan

    print("Synthetic modalities:", {k:v.shape for k,v in modalities.items()})


## 3. Sample alignment + z-score normalization


In [4]:
omics_keys = list(files.keys())
all_samples = sorted(set().union(*[modalities[k].index for k in omics_keys]))

def align(df): 
    return df.reindex(all_samples)

modalities_aligned = {k: align(modalities[k]) for k in omics_keys}

def zscore(df):
    mu = df.mean(axis=0, skipna=True)
    sd = df.std(axis=0, skipna=True).replace(0, 1.0)
    return (df - mu) / sd

modalities_aligned = {k: zscore(v) for k,v in modalities_aligned.items()}

X = {k: v.to_numpy(dtype=np.float32) for k,v in modalities_aligned.items()}
print({k: X[k].shape for k in X})


{'rna': (32, 19660), 'rppa': (32, 158), 'meth': (32, 20114)}


## 4. Dataset for block-wise masked-modality training


In [5]:
dataset = MultiOmicsDataset(X, mask_prob=0.3)
loader = DataLoader(dataset, batch_size=16, shuffle=True)


## 6. Training


In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
in_dims = {m: X[m].shape[1] for m in X}

model = OmicsMind(in_dims, z_dim=64).to(device)
opt = torch.optim.Adam(model.parameters(), lr=1e-3)

def to_tensor_batch(batch):
    sample, observed, train_mask = batch
    batch_x = {m: torch.tensor(sample[m], device=device) for m in sample}
    observed_t = {m: torch.tensor(observed[m], device=device) for m in observed}
    train_mask_t = {m: torch.tensor(train_mask[m], device=device) for m in train_mask}
    return batch_x, observed_t, train_mask_t

epochs = 200
for ep in range(1, epochs+1):
    model.train()
    total = total_rec = total_kl = 0.0
    for batch in loader:
        bx, obs_t, mask_t = to_tensor_batch(batch)
        refined_xhat, mus, logvars = model(bx, mask_t)
        loss, rec, kl = omicsmind_loss(bx, refined_xhat, mus, logvars, obs_t, mask_t, only_masked=False)

        opt.zero_grad()
        loss.backward()
        opt.step()

        total += loss.item(); total_rec += rec.item(); total_kl += kl.item()

    if ep % 5 == 0 or ep == 1:
        print(f"Epoch {ep:03d} | loss={total/len(loader):.4f} rec={total_rec/len(loader):.4f} kl={total_kl/len(loader):.4f}")


Epoch 001 | loss=2.9447 rec=2.9311 kl=13.6112
Epoch 005 | loss=2.9359 rec=2.8949 kl=41.0143
Epoch 010 | loss=2.8495 rec=2.8280 kl=21.4989
Epoch 015 | loss=2.8688 rec=2.8453 kl=23.5506
Epoch 020 | loss=2.7160 rec=2.6866 kl=29.3792
Epoch 025 | loss=2.5942 rec=2.5509 kl=43.2452
Epoch 030 | loss=2.6269 rec=2.5500 kl=76.9124
Epoch 035 | loss=2.4293 rec=2.3491 kl=80.1967
Epoch 040 | loss=2.3849 rec=2.2679 kl=116.9851
Epoch 045 | loss=2.3041 rec=2.2175 kl=86.5792
Epoch 050 | loss=2.1833 rec=2.0689 kl=114.4531
Epoch 055 | loss=2.1107 rec=1.9836 kl=127.1076
Epoch 060 | loss=2.1927 rec=2.0817 kl=111.0127
Epoch 065 | loss=2.0562 rec=1.9294 kl=126.8001
Epoch 070 | loss=2.1994 rec=2.0129 kl=186.5778
Epoch 075 | loss=2.0266 rec=1.8758 kl=150.8167
Epoch 080 | loss=1.8170 rec=1.6806 kl=136.4053
Epoch 085 | loss=1.8349 rec=1.7157 kl=119.1924
Epoch 090 | loss=1.7214 rec=1.5931 kl=128.2557
Epoch 095 | loss=1.7098 rec=1.5807 kl=129.0930
Epoch 100 | loss=1.4788 rec=1.3193 kl=159.5007
Epoch 105 | loss=1.657

## 9. Full imputation and CSV export


In [7]:
impute_data_dir = "./data/GBM_raw"  # Change this! Points to the dataset for imputation.

print("="*60)
print("LOADING DATA TO IMPUTE (ORDER-PRESERVING)")
print("="*60)
print(f"Data directory: {impute_data_dir}\n")

# ---- Load raw data to be imputed (preserving original row order) ----
impute_modalities = {}
for k, fn in files.items():
    p = os.path.join(impute_data_dir, fn)
    if os.path.exists(p):
        df = load_omics_table(p)

        # ⚠ Important: do NOT sort or union — preserve the file’s original row order
        impute_modalities[k] = df.copy()

        print(f"✓ {k} loaded from {fn}, shape: {df.shape}")
        print(f"  → Block-missing rows:  {df.isna().all(axis=1).sum()}")
        print(f"  → Total missing values: {df.isna().sum().sum()}")
    else:
        raise FileNotFoundError(f"Required file not found: {p}")


# ---- Merge sample IDs from all modalities while preserving original order ----
def union_preserve_order(list_of_indices):
    seen = set()
    output = []
    for idx_list in list_of_indices:
        for x in idx_list:
            if x not in seen:
                output.append(x)
                seen.add(x)
    return output

# Use raw indices from all modalities, without sorting
impute_samples = union_preserve_order([
    impute_modalities[k].index.tolist() for k in omics_keys
])

print("\nTotal samples to impute:", len(impute_samples))
print("Sample order preserved.\n")


# ---- Align all modalities to the same sample order (no sorting) ----
impute_aligned = {
    k: impute_modalities[k].reindex(impute_samples)
    for k in omics_keys
}


# ---- Extract normalization parameters from training data ----
print("="*60)
print("EXTRACTING TRAINING NORMALIZATION PARAMS")
print("="*60)

normalization_params = {}
for k in omics_keys:
    train_df = modalities[k]  # Training data modality (original scale)
    mu = train_df.mean(axis=0, skipna=True)
    sd = train_df.std(axis=0, skipna=True).replace(0, 1.0)
    normalization_params[k] = {'mean': mu, 'std': sd}
    print(f"{k}: mean/std loaded (shape {mu.shape})")


# ---- Normalize imputation data using training parameters (order preserved) ----
print("\nApplying training normalization to imputation data...")

impute_normalized = {}
for k in omics_keys:
    mu = normalization_params[k]['mean']
    sd = normalization_params[k]['std']
    impute_normalized[k] = (impute_aligned[k] - mu) / sd
    print(f"✓ {k} normalized using training params")


# ---- Convert to numpy arrays ----
X_impute = {
    k: impute_normalized[k].to_numpy(dtype=np.float32)
    for k in omics_keys
}

print("\nData normalization complete.\n")


# ---- Build dataset without altering sample order ----
model.eval()

class ImputationDataset(Dataset):
    def __init__(self, X_dict):
        self.mods = list(X_dict.keys())
        self.X = X_dict
        self.n = next(iter(X_dict.values())).shape[0]
    
    def __len__(self):
        return self.n
    
    def __getitem__(self, idx):
        sample, observed = {}, {}
        for m in self.mods:
            x_m = self.X[m][idx]
            obs_m = not np.all(np.isnan(x_m))   # block missing?
            observed[m] = obs_m
            sample[m] = np.nan_to_num(x_m, nan=0.0)
        return sample, observed, idx


impute_dataset = ImputationDataset(X_impute)
impute_loader = DataLoader(impute_dataset, batch_size=16, shuffle=False)


# ---- Perform imputation ----
print("="*60)
print("IMPUTING MISSING MODALITIES (ORDER PRESERVED)")
print("="*60)

imputed_X = {m: X_impute[m].copy() for m in omics_keys}

with torch.no_grad():
    for sample, observed, indices in tqdm(impute_loader, desc="Imputation"):
        batch_x = {m: torch.tensor(sample[m], device=device) for m in sample}
        observed_t = {m: torch.tensor(observed[m], device=device) for m in observed}

        # Mask: 1 = observed; 0 = block-missing
        infer_mask = {m: observed_t[m].float() for m in observed_t}

        refined_xhat, _, _ = model(batch_x, infer_mask)

        # Write predictions back into imputed_X
        for i, global_idx in enumerate(indices):
            for m in omics_keys:
                if not observed[m][i]:
                    imputed_X[m][global_idx] = refined_xhat[m][i].cpu().numpy()


# ---- Denormalize results back to original scale ----
print("\nDenormalizing imputed values...")

imputed_original_scale = {}
for m in omics_keys:
    mu = normalization_params[m]['mean'].values
    sd = normalization_params[m]['std'].values
    restored = imputed_X[m] * sd + mu

    imputed_original_scale[m] = pd.DataFrame(
        restored,
        index=impute_samples,                  # ⚠ Do not reorder
        columns=impute_aligned[m].columns      # Column order matches training data
    )

    print(f"{m}: restored to original scale\n")


# ---- Merge original and imputed values (overwrite ONLY NaNs) ----
print("="*60)
print("MERGING ORIGINAL AND IMPUTED VALUES")
print("="*60)

final_data = {}

for m in omics_keys:
    original = impute_aligned[m]    # Original data (with NaNs)
    imputed  = imputed_original_scale[m]

    # Keep original observed values; fill only NaNs
    final = original.combine_first(imputed)

    final_data[m] = final

    print(f"{m}: NaNs before = {original.isna().sum().sum()}, after = {final.isna().sum().sum()}")


# ---- Export results ----
output_dir = "./imputed_results/GBM_imputed"
os.makedirs(output_dir, exist_ok=True)

print("\nExporting final CSVs...")

for m in omics_keys:
    out = os.path.join(output_dir, f"{m}.csv")
    final_data[m].to_csv(out)
    print(f"✓ Saved {out}  (shape {final_data[m].shape})")

print("\nAll imputation complete. Original sample order preserved!")


LOADING DATA TO IMPUTE (ORDER-PRESERVING)
Data directory: ./data/GBM_raw

✓ rna loaded from rna.csv, shape: (595, 19660)
  → Block-missing rows:  443
  → Total missing values: 8709380
✓ rppa loaded from rppa.csv, shape: (595, 158)
  → Block-missing rows:  363
  → Total missing values: 57442
✓ meth loaded from meth.csv, shape: (595, 20114)
  → Block-missing rows:  457
  → Total missing values: 9193530

Total samples to impute: 595
Sample order preserved.

EXTRACTING TRAINING NORMALIZATION PARAMS
rna: mean/std loaded (shape (19660,))
rppa: mean/std loaded (shape (158,))
meth: mean/std loaded (shape (20114,))

Applying training normalization to imputation data...
✓ rna normalized using training params
✓ rppa normalized using training params
✓ meth normalized using training params

Data normalization complete.

IMPUTING MISSING MODALITIES (ORDER PRESERVED)


Imputation: 100%|██████████████████████████████████████████████████████████████████████| 38/38 [00:00<00:00, 92.91it/s]



Denormalizing imputed values...
rna: restored to original scale

rppa: restored to original scale

meth: restored to original scale

MERGING ORIGINAL AND IMPUTED VALUES
rna: NaNs before = 8709380, after = 0
rppa: NaNs before = 57442, after = 88
meth: NaNs before = 9193530, after = 1432

Exporting final CSVs...
✓ Saved ./imputed_results/GBM_imputed\rna.csv  (shape (595, 19660))
✓ Saved ./imputed_results/GBM_imputed\rppa.csv  (shape (595, 158))
✓ Saved ./imputed_results/GBM_imputed\meth.csv  (shape (595, 20114))

All imputation complete. Original sample order preserved!
